# Model Training

In [1]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
import os

def image_gen_w_aug(train_parent_directory, test_parent_directory):
    
    train_datagen = ImageDataGenerator(rescale=1/255,
                                      rotation_range = 30,  
                                      zoom_range = 0.2, 
                                      width_shift_range=0.1,  
                                      height_shift_range=0.1,
                                      validation_split = 0.15)
    
    test_datagen = ImageDataGenerator(rescale=1/255)
    
    train_generator = train_datagen.flow_from_directory(train_parent_directory,
                                                       target_size = (75,75),
                                                       batch_size = 214,
                                                       class_mode = 'categorical',
                                                       subset='training')
    
    val_generator = train_datagen.flow_from_directory(train_parent_directory,
                                                          target_size = (75,75),
                                                          batch_size = 37,
                                                          class_mode = 'categorical',
                                                          subset = 'validation')
    
    test_generator = test_datagen.flow_from_directory(test_parent_directory,
                                                     target_size=(75,75),
                                                     batch_size = 37,
                                                     class_mode = 'categorical')
    
    return train_generator, val_generator, test_generator


def model_output_for_TL (pre_trained_model, last_output):

    x = Flatten()(last_output)
    
    # Dense hidden layer
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.2)(x)
    
    # Output neuron. 
    x = Dense(3, activation='softmax')(x)
    
    model = Model(pre_trained_model.input, x)
    
    return model


train_dir = os.path.join('./rps/datasets/train/')
test_dir = os.path.join('./rps/datasets/test/')

train_generator, validation_generator, test_generator = image_gen_w_aug(train_dir, test_dir)

pre_trained_model = InceptionV3(input_shape = (75, 75, 3), 
                                include_top = False, 
                                weights = 'imagenet')

for layer in pre_trained_model.layers:
  layer.trainable = False

last_layer = pre_trained_model.get_layer('mixed3')
last_output = last_layer.output

model_TL = model_output_for_TL(pre_trained_model, last_output)
model_TL.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

history_TL = model_TL.fit(
      train_generator,
      epochs=5,
      verbose=1,
      validation_data = validation_generator)

tf.keras.models.save_model(model_TL,'./models/my_model.keras')



c:\Users\bryon\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\export\tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


Found 2142 images belonging to 3 classes.
Found 378 images belonging to 3 classes.
Found 33 images belonging to 3 classes.
Epoch 1/5
11/11 ━━━━━━━━━━━━━━━━━━━━ 9s 594ms/step - accuracy: 0.4897 - loss: 5.9200 - val_accuracy: 0.5767 - val_loss: 2.2362
Epoch 2/5
11/11 ━━━━━━━━━━━━━━━━━━━━ 6s 502ms/step - accuracy: 0.7353 - loss: 1.1682 - val_accuracy: 0.7540 - val_loss: 0.6383
Epoch 3/5
11/11 ━━━━━━━━━━━━━━━━━━━━ 6s 515ms/step - accuracy: 0.8926 - loss: 0.3823 - val_accuracy: 0.9021 - val_loss: 0.2576
Epoch 4/5
11/11 ━━━━━━━━━━━━━━━━━━━━ 6s 515ms/step - accuracy: 0.9510 - loss: 0.1357 - val_accuracy: 0.9762 - val_loss: 0.0782
Epoch 5/5
11/11 ━━━━━━━━━━━━━━━━━━━━ 6s 503ms/step - accuracy: 0.9781 - loss: 0.0623 - val_accuracy: 0.9603 - val_loss: 0.0852


# Inference

In [9]:
from PIL import Image, ImageOps
import tensorflow as tf
import cv2
import numpy as np

frame = None

model = tf.keras.models.load_model('./models/my_model.keras')

def import_and_predict(image_data):
    
        image = ImageOps.fit(image_data, (75, 75), Image.Resampling.LANCZOS)
        image = image.convert('RGB')
        image = np.asarray(image)
        image = (image.astype(np.float32) / 255.0)

        img_reshape = image[np.newaxis,...]

        prediction = model.predict(img_reshape)
        
        return prediction


cap = cv2.VideoCapture(0)

if (cap.isOpened()):
    print("Camera OK")
else:
    cap.open()

for _ in range(3000):
    ret, original = cap.read()
    if not ret:
        break

    frame = cv2.resize(original, (224, 224))
    image = Image.fromarray(cv2.cvtColor(original, cv2.COLOR_BGR2RGB))

    # Display the predictions
    prediction = import_and_predict(image)
    #print(prediction)

    if np.argmax(prediction) == 0:
        predict="It is a paper!"
    elif np.argmax(prediction) == 1:
        predict="It is a rock!"
    else:
        predict="It is a scissor!"
    
    cv2.putText(original, predict, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
    cv2.imshow("Classification", original)

    if (cv2.waitKey(1) & 0xFF == ord('q')):
        break

cap.release()
frame = None
cv2.destroyAllWindows()


Camera OK
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 532ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1

KeyboardInterrupt: 

In [ ]:
# code imma take reference frm

def visualize_predictions_on_all_images(images_folder: str, class_names: list[str]) -> None:
    """
    Reads all images in the folder, predicts bounding boxes using the loaded YOLO model,
    and visualizes the images with predicted boxes and class names.

    Args:
        images_folder: path to folder containing images
        class_names: list of class names
    """
    all_images = [f for f in os.listdir(images_folder) if f.lower().endswith(('.jpg', '.png', '.jpeg'))]

    for img_name in all_images:
        img_path = os.path.join(images_folder, img_name)

        # Load image
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        # Predict
        results = best_model.predict(img_path, conf=0.25, verbose=False)
        boxes = results[0].boxes.xyxy.cpu().numpy()
        classes = results[0].boxes.cls.cpu().numpy()

        # Draw predicted boxes
        for box, cls in zip(boxes, classes):
            x1, y1, x2, y2 = map(int, box)
            cv2.rectangle(img, (x1, y1), (x2, y2), (255, 0, 0), 2)
            cv2.putText(img, class_names[int(cls)], (x1, y1 - 5),
                        cv2.FONT_HERSHEY_SIMPLEX, 1.2, (255, 0, 0), 1)

        # Show image
        plt.figure(figsize=(10, 8))
        plt.imshow(img)
        plt.axis('off')
        plt.title(f"Predictions: {img_name}", fontsize=12)
        plt.show()